# Libraries

In [38]:
import pandas as pd
import pickle
from scipy.stats import poisson
from bs4 import BeautifulSoup
import requests
from string import ascii_uppercase as alphabet

# Get Games

In [39]:
def get_matches_WWC_2023():
    web = 'https://en.wikipedia.org/wiki/2023_FIFA_Women%27s_World_Cup'
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')
    matches = soup.find_all('div', class_='footballbox')

    home = []
    score = []
    away = []

    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())

    dict_football = {'home': home, 'score': score, 'away': away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = 2023
    return df_football

# WWC games in 2023
df_future = get_matches_WWC_2023()

In [40]:
df_future['home'] = df_future['home'].str.strip()
df_future['away'] = df_future['away'].str.strip()

In [41]:
all_tables = pd.read_html('https://en.wikipedia.org/wiki/2023_FIFA_Women%27s_World_Cup')
all_tables[9]

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,New Zealand (H),0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Norway,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Philippines,0,0,0,0,0,0,0,0,NaN
3,4,Switzerland,0,0,0,0,0,0,0,0,NaN


In [42]:
# A -> H
# 7*8 + 9 = 65
all_tables = pd.read_html('https://en.wikipedia.org/wiki/2023_FIFA_Women%27s_World_Cup')
for i in range(9,65,7):
    df = all_tables[i]
    df.rename(columns={df.columns[1]:'Team'}, inplace=True)
    df.pop('Qualification')

In [43]:
all_tables = pd.read_html('https://en.wikipedia.org/wiki/2023_FIFA_Women%27s_World_Cup')

dict_table = {}
for letter, i in zip(alphabet, range(9,65,7)):
    df = all_tables[i]
    df.rename(columns={df.columns[1]:'Team'}, inplace=True)
    df.pop('Qualification')
    dict_table[f'Group {letter}'] = df

In [44]:
dict_table['Group H']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Germany,0,0,0,0,0,0,0,0
1,2,Morocco,0,0,0,0,0,0,0,0
2,3,Colombia,0,0,0,0,0,0,0,0
3,4,South Korea,0,0,0,0,0,0,0,0


# Prediction

In [45]:
df_historical_games = pd.read_csv('df_historical_data_kaggle.csv')

In [46]:
df_home_team = df_historical_games[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away_team = df_historical_games[['AwayTeam', 'HomeGoals', 'AwayGoals']]

df_home_team = df_home_team.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away_team = df_away_team.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

df_team_strength = pd.concat([df_home_team, df_away_team], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Albania,0.945946,2.891892
Algeria,1.552632,1.657895
American Samoa,0.000000,7.000000
Andorra,0.500000,5.500000
Angola,1.333333,1.733333
...,...,...
West Germany,1.000000,2.500000
Western Australia,0.666667,1.666667
Yugoslavia,0.000000,3.000000


In [47]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11):
            for y in range(0, 11):
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p

        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [48]:
df_future_group_48 = df_future[:48].copy()
df_future_knockout = df_future[48:56].copy()
df_future_quarter = df_future[56:60].copy()
df_future_semi = df_future[60:62].copy()
df_future_final = df_future[62:].copy()

## Group stage

In [49]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_future_group_6 = df_future_group_48[df_future_group_48['home'].isin(teams_in_group)]
    for index, row in df_future_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [62]:
dict_table['Group F']

,Team,Pts
0,Brazil,7.0
1,France,6.0
2,Jamaica,3.0
3,Panama,1.0


## Functions to get winners

In [51]:
def get_winner(df_future_updated):
    for index, row in df_future_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_future_updated.loc[index, 'winner'] = winner
    return df_future_updated

In [52]:
def update_table(df_future_round_1, df_future_round_2):
    for index, row in df_future_round_1.iterrows():
        winner = df_future_round_1.loc[index, 'winner']
        match = df_future_round_1.loc[index, 'score']
        df_future_round_2.replace({f'Winner {match}':winner}, inplace=True)
    df_future_round_2['winner'] = '?'
    return df_future_round_2

## Round of 16

In [53]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_future_knockout.replace({f'Winner {group}':group_winner,
                                 f'Runner-up {group}':runners_up}, inplace=True)

df_future_knockout['winner'] = '?'

In [54]:
get_winner(df_future_knockout)

,home,score,away,year,winner
48,Norway,Match 49,Japan,2023,Norway
49,Spain,Match 50,Switzerland,2023,Spain
50,United States,Match 51,Italy,2023,United States
51,Sweden,Match 52,Netherlands,2023,Sweden
52,England,Match 54,Nigeria,2023,England
53,Canada,Match 53,Denmark,2023,Denmark
54,Germany,Match 56,France,2023,Germany
55,Brazil,Match 55,South Korea,2023,Brazil


In [55]:
update_table(df_future_knockout, df_future_quarter)

,home,score,away,year,winner
56,Norway,Match 57,United States,2023,?
57,Spain,Match 58,Sweden,2023,?
58,Denmark,Match 59,Brazil,2023,?
59,England,Match 60,Germany,2023,?


## Quarter-finals

In [56]:
get_winner(df_future_quarter)

,home,score,away,year,winner
56,Norway,Match 57,United States,2023,United States
57,Spain,Match 58,Sweden,2023,Sweden
58,Denmark,Match 59,Brazil,2023,Brazil
59,England,Match 60,Germany,2023,Germany


In [57]:
update_table(df_future_quarter, df_future_semi)

,home,score,away,year,winner
60,United States,Match 61,Sweden,2023,?
61,Brazil,Match 62,Germany,2023,?


## Semi-finals

In [58]:
get_winner(df_future_semi)

,home,score,away,year,winner
60,United States,Match 61,Sweden,2023,United States
61,Brazil,Match 62,Germany,2023,Germany


In [59]:
update_table(df_future_semi, df_future_final)

,home,score,away,year,winner
62,Loser Match 61,Match 63,Loser Match 62,2023,?
63,United States,Match 64,Germany,2023,?


## Final

In [60]:
get_winner(df_future_final)

,home,score,away,year,winner
62,Loser Match 61,Match 63,Loser Match 62,2023,Loser Match 62
63,United States,Match 64,Germany,2023,United States
